## Classificação

In [71]:
#imports
import numpy as np
from sklearn import svm, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE, f_classif

Load

In [72]:
features = np.load("processed/extracted_features.npy").swapaxes(1,2)
features.shape

(2, 924, 18)

In [73]:
# protocolo
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# criação do vetor de labels final
y = labels_num * int(features.shape[1] / len(labels_num))
len(y)

924

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [74]:
# aplicando normalização

for i in range(len(features)):
  features[i] = StandardScaler().fit_transform(features[i])

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

In [77]:
# aplicando seleção de características
X_all = list()

threshold = 1
k = 9

for i in range(len(features)):
  # SELECT K BEST
  kb = SelectKBest(f_classif, k=k).fit_transform(features[i], y)
  X_all.append((f'p{i}-kBest', kb))
  print(kb.shape)

  # VARIANCE THRESHOLD
  vt = VarianceThreshold(threshold).fit_transform(features[i])
  X_all.append((f'p{i}-vTresh', vt))
  print(vt.shape)

  # RECURSIVE FEATURE ELIMINATION
  rfe = RFE(svm.SVC(kernel="linear")).fit_transform(features[i], y)
  X_all.append((f'p{i}-RFE', rfe))
  print(rfe.shape)

/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(924, 9) (924, 18)
(924, 2) (924, 18)
(924, 9) (924, 18)
(924, 9) (924, 18)
(924, 3) (924, 18)


/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 1] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/caio/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


(924, 9) (924, 18)


*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [ ]:
# aplicando a classificação

for X in X_all:
    print(f'{X[0]} --------------------------------------------')
    # Divisão Treino/Teste
    x_train, x_test, y_train, y_test = train_test_split(X[1], y, test_size=0.2, random_state=42)
    print('Treino >', x_train.shape, len(y_train))
    print('Teste >', x_test.shape, len(y_test))
    
    # Grid search
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': [0.1, 1, 10],
        'degree': [2, 3, 4],
        'coef0': [0, 0.5, 1],
        'tol': [1e-4, 1e-5, 1e-6]
    }
    grid = GridSearchCV(svm.SVC(), param_grid, cv=10)
    grid.fit(x_train, y_train)

    # Predição
    y_pred = grid.best_estimator_.predict(x_test)
    print('Acurácia >', metrics.accuracy_score(y_test, y_pred))
    # print(metrics.classification_report(y_test, y_pred, digits=5))

p0-vTresh --------------------------------------------
Treino > (739, 16) 739
Teste > (185, 16) 185


KeyboardInterrupt: 